In [ ]:
import os
import numpy as np
import pandas as pd
import pickle

# import warnings
# warnings.ignore('all')

import matplotlib.pyplot as plt
%matplotlib inline

from skimage.measure import compare_psnr
from sklearn.preprocessing import StandardScaler

import utils
from modules import RotationLSH, IdentityLSH, PcaLSH, RandomForestClustRegressor, PictureResolver

In [ ]:
# CONFIG

n_samples=50000
s=1/2
n_estimators=10
max_depth=15
max_features=10

augmented = False
lsh = IdentityLSH()
binary = True        # makes sense to use only for RotationLSH
reg_type = 'ridge'

In [ ]:
def crop_image(img_array, s):
    """Crop image so that downscale-upscaling works well"""
    img_array = img_array.copy()
    shape = img_array.shape
    new_shape = tuple(dim - (dim % int(1/s)) for dim in shape) # so that the HR and LR images have same size
    img_array = img_array[0:new_shape[0], 0:new_shape[1]]
    return img_array

In [ ]:
data_dir = '../data/Set14'
train_files = ['barbara.png', 'bridge.png', 'comic.png', 'pepper.png', 'man.png']

def generate_trainset(n_samples, s, seed=7):
    # patch_size is (9,9)
    
    np.random.seed(seed)
    X = []
    Y = []
    for file in train_files:
        path = os.path.join(data_dir, file)
        img_h = utils.load_image(path)
        img_h = crop_image(img_h, s)
        new_shape = img_h.shape
        
        img_l = utils.resize(utils.resize(img_h, s), 1/s) # downsample -> upsample

        features = utils.extract_features(img_l, augmented=augmented)
        img_out_delta = img_h - img_l
        assert img_h.shape == img_l.shape == img_out_delta.shape == new_shape

        x_ind = np.random.randint(0, new_shape[0]-9, size=n_samples//len(train_files))
        y_ind = np.random.randint(0, new_shape[1]-9, size=n_samples//len(train_files))
        for x, y in zip(x_ind, y_ind):
            feat_patch = features[x:x+9, y:y+9]
            delta_patch = img_out_delta[x:x+9, y:y+9]
            X.append(feat_patch.flatten())
            Y.append(delta_patch.flatten())

    X = np.array(X)
    Y = np.array(Y)
    return X, Y

In [ ]:
X, Y = generate_trainset(
    n_samples=n_samples,
    s=s,
    seed=7
)

X = np.random.randn(500,324)
Y = np.random.randn(500,81)
print(X.shape)
print(Y.shape)

In [ ]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
lsh.fit(X)
X_comp = lsh.transform(X, binary=binary)

## Evaluate

In [ ]:
df = pd.DataFrame(columns=os.listdir(data_dir))

row_num = 0

for min_samples_leaf in [128, 256]:
    for alpha in [0.1, 0.5, 1, 2, 3, 4]:
        df.loc[row_num, 'min_samples_leaf'] = min_samples_leaf
        df.loc[row_num, 'reg_type'] = reg_type
        df.loc[row_num, 'alpha'] = alpha

        rf = RandomForestClustRegressor(
            n_estimators=n_estimators,
            max_depth=max_depth,
            n_jobs=None,
            max_features=max_features,
            min_samples_leaf=min_samples_leaf,
            reg_type=reg_type,
            alpha=alpha
        )
        rf.fit(X, X_comp, Y)

        for path_test in os.listdir(data_dir):
            img_test = utils.load_image(os.path.join(data_dir,path_test))
            img_test = crop_image(img_test, s)
            x = utils.resize(utils.resize(img_test, s), 1/s)

            pr = PictureResolver(rf, lsh, scaler)
            y = pr.resolve(x, augmented=augmented)

            nans = ~np.isnan(y)
            y = y[nans.any(axis=1), :]
            img_test = img_test[nans.any(axis=1), :]
            y = y[:, nans.any(axis=0)]
            img_test = img_test[:, nans.any(axis=0)]

            df.loc[row_num, path_test] = compare_psnr(img_test, y, np.max(img_test) - np.min(img_test))

        row_num += 1

In [ ]:
df

In [ ]:
#### CHANGE NAME TO STH MEANINGFUL

df.to_csv('identity_ridge_augmented=False.csv')

In [ ]:
a = np.arange(-2,7).reshape(3,3)
a.nonzero()